In [ ]:
!pip3 install numpy
!pip3 install pandas
!pip3 install scikit-learn
!pip3 install rdkit
!pip3 install torcheval

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdchem
from rdkit.Chem import AllChem
import re


Chiral = {"CHI_UNSPECIFIED":0,  "CHI_TETRAHEDRAL_CW":1, "CHI_TETRAEDRAL_CCW":2, "CHI_OTHER":3}
Hybridization = {"UNSPECIFIED":0, "S":1, "SP":2, "SP2":3, "SP3":4, "SP3D":5, "SP3D2":6, "OTHER":7}

 # feature vector size
atomInfo = 210
structInfo = 21
lensize= atomInfo + structInfo

H_Vector = [0]*atomInfo
H_Vector[0]= 1

# 소문자 일치 판정
lowerReg = re.compile(r'^[a-z]+$')
def islower(s):
    return lowerReg.match(s) is not None

# 대문자 일치 판정
upperReg = re.compile(r'^[A-Z]+$')
def isupper(s):
    return upperReg.match(s) is not None

# atom 정보
def calc_atom_feature(atom):

    if atom.GetSymbol() == 'H':   feature = [1,0,0,0,0]
    elif atom.GetSymbol() == 'C': feature = [0,1,0,0,0]
    elif atom.GetSymbol() == 'O': feature = [0,0,1,0,0]
    elif atom.GetSymbol() == 'N': feature = [0,0,0,1,0]
    else: feature = [0,0,0,0,1]

    feature.append(atom.GetTotalNumHs()/8)
    feature.append(atom.GetTotalDegree()/4)
    feature.append(atom.GetFormalCharge()/8)
    feature.append(atom.GetTotalValence()/8)
    feature.append(atom.IsInRing()*1)
    feature.append(atom.GetIsAromatic()*1)

    f =  [0]*(len(Chiral)-1)
    if Chiral.get(str(atom.GetChiralTag()), 0) != 0:
        f[Chiral.get(str(atom.GetChiralTag()), 0)] = 1
    feature.extend(f)

    f =  [0]*(len(Hybridization)-1)
    if Hybridization.get(str(atom.GetHybridization()), 0) != 0:
        f[Hybridization.get(str(atom.GetHybridization()), 0)] = 1
    feature.extend(f)

    return(feature)


def calc_structure_feature(c,flag,label):
    feature = [0]*structInfo

    if c== '(' :
        feature[0] = 1
        flag = 0
    elif c== ')' :
        feature[1] = 1
        flag = 0
    elif c== '[' :
        feature[2] = 1
        flag = 0
    elif c== ']' :
        feature[3] = 1
        flag = 0
    elif c== '.' :
        feature[4] = 1
        flag = 0
    elif c== ':' :
        feature[5] = 1
        flag = 0
    elif c== '=' :
        feature[6] = 1
        flag = 0
    elif c== '#' :
        feature[7] = 1
        flag = 0
    elif c== '\\':
        feature[8] = 1
        flag = 0
    elif c== '/' :
        feature[9] = 1
        flag = 0
    elif c== '@' :
        feature[10] = 1
        flag = 0
    elif c== '+' :
        feature[11] = 1
        flag = 1
    elif c== '-' :
        feature[12] = 1
        flag = 1
    elif c.isdigit() == True:
        if flag == 0:
            if c in label:
                feature[20] = 1
            else:
                label.append(c)
                feature[19] = 1
        else:
            feature[int(c)-1+12] = 1
            flag = 0
    return(feature,flag,label)


def calc_featurevector(mol, smiles,atomsize):
    flag = 0
    label = []
    molfeature=[]
    idx = 0
    j = 0

    for c in smiles:
        if islower(c) == True: continue
        elif isupper(c) == True:
            if c == 'H':
                molfeature.extend(H_Vector)
            else:
                molfeature.extend(calc_atom_feature(rdchem.Mol.GetAtomWithIdx(mol, idx)))
                idx = idx + 1
            molfeature.extend([0]*structInfo)
            j = j +1

        else:
            molfeature.extend([0]*atomInfo)
            f,flag,label = calc_structure_feature(c,flag,label)
            molfeature.extend(f)
            j = j +1

    # 0-Padding
    molfeature.extend([0]*(atomsize-j)*lensize)
    return(molfeature)


def mol_to_feature(mol,n,atomsize):
    try: defaultSMILES = Chem.MolToSmiles(mol, kekuleSmiles=False, isomericSmiles=True, rootedAtAtom=int(n))
    except: defaultSMILES = Chem.MolToSmiles(mol, kekuleSmiles=False, isomericSmiles=True)
    try: isomerSMILES = Chem.MolToSmiles(mol, kekuleSmiles=True, isomericSmiles=True, rootedAtAtom=int(n))
    except: isomerSMILES = Chem.MolToSmiles(mol, kekuleSmiles=True, isomericSmiles=True)
    return calc_featurevector(Chem.MolFromSmiles(defaultSMILES), isomerSMILES,atomsize)

def mol_to_allSMILESfeature(mol, atomsize):
    idx, features =0,  []
    while idx < mol.GetNumAtoms():
        try: defaultSMILES = Chem.MolToSmiles(mol, kekuleSmiles=False, isomericSmiles=True, rootedAtAtom=int(idx))
        except: break
        isomerSMILES = Chem.MolToSmiles(mol, kekuleSmiles=True, isomericSmiles=True, rootedAtAtom=int(idx))
        features.append(calc_featurevector(Chem.MolFromSmiles(defaultSMILES), isomerSMILES,atomsize))
        idx = idx + 1
    return(features)

#-------------------------------------------------------------
 # Sigmoid function definition
def strong_sigmoid(x):
    return 1*(x >=0)

#-------------------------------------------------------------
 # detaset function definition
def random_list(x, seed=0):
    np.random.seed(seed)
    np.random.shuffle(x)

def data_boostDataset(P,N,boost=1,seed=0):
    random_list(P,seed)
    random_list(N,seed)
    T = [0]*len(N)+ [1]*(len(P)*boost)
    for i in range(boost):N.extend(P)
    random_list(N,seed)
    random_list(T,seed)
    return N, T

In [ ]:
from rdkit import Chem
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torcheval.metrics import BinaryAccuracy
from torch.autograd import Variable
from torch.utils.data import DataLoader # 학습 및 배치로 모델에 넣어주기 위한 툴

#import torch.nn.init as init # 텐서에 초기값을 줌
#import torchvision.datasets as datasets # 이미지 데이터셋 집합체
#import torchvision.transforms as transforms # 이미지 변환 툴

import numpy as np
import matplotlib.pyplot as plt

transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

trainset =
testset =

batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

class SCFPModel(nn.Module):
    def __init__(self, atomsize, lensize, k1, s1, f1, k2, s2, k3, s3, f3, k4, s4, n_hid, n_out):

        # atomsize, lenseize = size of feature matrix
        # k1, s1, f1 = window-size, stride-step, No. of filters of first convolution layer
        # k2, s2 = window-size, stride-step of first max-pooling layer
        # k3, s3, f3 = window-size, stride-step, No. of filters of second convolution layer
        # k4, s4 = window-size, stride-step of second max-pooling layer

        super().__init__()
        conv1 = nn.Conv2D(1, f1, (k1, lensize), stride=s1, pad = (k1//(2,0))),
        bn1 = nn.BatchNorm2d(f1),
        conv2 = nn.Conv2D(f1, f3, (k3, 1), stride=s3, pad = (k3//(2,0))),
        bn2 = nn.BatchNorm2d(f3),
        fc3 = nn.Linear(None, n_hid),
        bn3 = nn.BatchNorm2d(n_hid),
        fc4 = nn.Linear(None, n_out)

        self.atomsize, self.lensize, self.n_out = atomsize, lensize, n_out
        self.k1, self.s1, self.f1, self.k2, self.s2, self.k3, self.s3, self.f3, self.k4, self.s4 = k1, s1, f1, k2, s2, k3, s3, f3, k4, s4
        self.l1 = (self.atomsize+(self.k1//2*2)-self.k1)//self.s1+1
        self.l2 = (self.l1+(self.k2//2*2)-self.k2)//self.s2+1
        self.l3 = (self.l2+(self.k3//2*2)-self.k3)//self.s3+1
        self.l4 = (self.l3+(self.k4//2*2)-self.k4)//self.s4+1

    def predict(self,x):
        h = nn.LeakyReLU(self.bn1(self.conv1(x))) # 1st conv
        h = nn.AvgPool2d(h, (self.k2,1), stride=self.s2, pad=(self.k2//(2,0))) # 1st pooling
        h = nn.LeakyReLU(self.bn2(self.conv2(h))) # 2nd conv
        h = nn.AvgPool2d(h, (self.k4,1), stride=self.s4, pad=(self.k4//(2,0))) # 2nd pooling
        h = nn.MaxPool2d(h, (self.l4,1)) # grobal max pooling, fingerprint
        h = self.fc3(h) # fully connected
        sr = 0.00001* np.mean(np.log(1 + h.data * h.data)) # sparse regularization
        h = nn.LeakyReLU(self.bn3(h))
        return self.fc4(h), sr

    def __call__(self, x, t):
        y, sr = self.predict(x)
        loss = nn.BCEWithLogitsLoss(y, t) + sr
        accuracy = nn.BinaryAccuracy(y, t)
        report({'loss': loss, 'accuracy': accuracy}, self)
        return loss

    def fingerprint(self,x):
        x = Variable(x.astype(np.float32).reshape(-1,1, self.atomsize, self.lensize))
        h = nn.LeakyReLU(self.bn1(self.conv1(x))) # 1st conv
        h = nn.AvgPool2d(h, (self.k2,1), stride=self.s2, pad=(self.k2//(2,0))) # 1st pooling
        h = nn.LeakyReLU(self.bn2(self.conv2(h))) # 2nd conv
        h = nn.AvgPool2d(h, (self.k3,1), stride=self.s3, pad=(self.k3//(2,0))) # 2nd pooling
        h = nn.MaxPool2d(h, (self.l4,1)) # grobal max pooling, fingerprint
        return h.data

    def layer1(self,x):
        x = Variable(x.astype(np.float32).reshape(-1,1, self.atomsize, self.lensize))
        h = self.bn1(self.conv1(x)) # 1st conv
        return h.data

    def pool1(self,x):
        x = Variable(x.astype(np.float32).reshape(-1,1, self.atomsize, self.lensize))
        h = nn.LeakyReLU(self.bn1(self.conv1(x))) # 1st conv
        h = nn.AvgPool2d(h, (self.k2,1), stride=self.s2, pad=(self.k2//(2,0))) # 1st pooling
        return h.data

    def layer2(self,x):
        x = Variable(x.astype(np.float32).reshape(-1,1, self.atomsize, self.lensize))
        h = nn.LeakyReLU(self.bn1(self.conv1(x))) # 1st conv
        h = nn.AvgPool2d(h, (self.k2,1), stride=self.s2, pad=(self.k2//(2,0))) # 1st pooling
        h = self.bn2(self.conv2(h)) # 2nd conv
        return h.data

    def pool2(self,x):
        x = Variable(x.astype(np.float32).reshape(-1,1, self.atomsize, self.lensize))
        h = nn.LeakyReLU(self.bn1(self.conv1(x))) # 1st conv
        h = nn.AvgPool2d(h, (self.k2,1), stride=self.s2, pad=(self.k2//(2,0))) # 1st pooling
        h = nn.LeakyReLU(self.bn2(self.conv2(h))) # 2nd conv
        h = nn.AvgPool2d(h, (self.k3,1), stride=self.s3, pad=(self.k3//(2,0))) # 2nd pooling
        return h.data

NameError: ignored

In [ ]:
model = SCFPModel()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

n_epochs = 300
for epoch in range(n_epochs):
    for inputs, labels in trainloader:
        # forward, backward, and then weight update
        y_pred = model(inputs)
        loss = loss_fn(y_pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    acc = 0
    count = 0
    for inputs, labels in testloader:
        y_pred = model(inputs)
        acc += (torch.argmax(y_pred, 1) == labels).float().sum()
        count += len(labels)
    acc /= count
    print("Epoch %d: model accuracy %.2f%%" % (epoch, acc*100))

torch.save(model.state_dict(), "SCFPmodel.pth")